# Move Bounds

Notebook on moving bounds for systems, while also calculating the the $\sigma$s

In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import tvsclib.utils as utils

from tvsclib.transformations.output_normal import OutputNormal
from tvsclib.transformations.reduction import Reduction

In [ ]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
#matrix = np.random.rand(12,12)
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-12)

system = MixedSystem(S).causal_system
utils.show_system(system)

First work on the casual part:

Require a output normal system, start with the lowest index and work up, while transforming to a input normal system

In [ ]:
def transform(stages,direction):
    k = len(stages)
    sigmas = []
    for i in range(1,len(stages)):#loop over inices of causal states
        if direction == "l":
            #move left:
            b = stages[i-1].B_matrix[:,-1:]
            U,s_l,Vt= np.linalg.svd(np.hstack([stages[i-1].A_matrix,stages[i-1].B_matrix[:,:-1]]),full_matrices=False)
            Us=U*s_l

            stages_l = [
                Stage(Vt[:,:stages[i-1].A_matrix.shape[1]],Vt[:,stages[i-1].A_matrix.shape[1]:],\
                        stages[i-1].C_matrix,stages[i-1].D_matrix[:,:-1]),
                Stage(stages[i].A_matrix@Us,np.hstack((stages[i].A_matrix@b,stages[i].B_matrix)),\
                        stages[i].C_matrix@Us,np.hstack((stages[i].C_matrix@b,stages[i].D_matrix)))
            ]

        if direction =="r"or direction=="n":
            #no move-> only make R_k input normal
            U,s,Vt= np.linalg.svd(np.hstack([stages[i-1].A_matrix,stages[i-1].B_matrix]),full_matrices=False)
            Us=U*s

            stages_n=[
                Stage(Vt[:,:stages[i-1].A_matrix.shape[1]],Vt[:,stages[i-1].A_matrix.shape[1]:],\
                        stages[i-1].C_matrix,stages[i-1].D_matrix),
                Stage(stages[i].A_matrix@Us,stages[i].B_matrix,\
                        stages[i].C_matrix@Us,stages[i].D_matrix)
            ]

        #move right -> base on non move
        if direction =="r":
            b = stages_n[1].B_matrix[:,:1]
            d = stages_n[1].D_matrix[:,:1]
            d_add = np.zeros((stages_n[0].D_matrix.shape[0],1))


            U,s_r,Vt= np.linalg.svd(np.block([[stages_n[1].A_matrix,b],
                                              [stages_n[1].C_matrix,d]]),full_matrices=False)
            Us=U*s_r
            stages_r=[
                 #Here the A and B are more complicated as we have to stack them
                Stage(Vt@(np.vstack([stages_n[0].A_matrix,np.zeros((1,stages_n[0].A_matrix.shape[1]))])),
                      Vt@(np.block([[stages_n[0].B_matrix,np.zeros((stages_n[0].B_matrix.shape[0],1))],
                                   [np.zeros(stages_n[0].B_matrix.shape[1]),np.eye(1)]])),
                      stages_n[0].C_matrix,np.hstack([stages_n[0].D_matrix,d_add])),

               Stage(Us[:stages_n[1].A_matrix.shape[0],:],stages_n[1].B_matrix[:,1:],\
                     Us[stages_n[1].A_matrix.shape[0]:,:],stages_n[1].D_matrix[:,1:])                 
            ]

        if direction == "l":
            stages[i-1]= stages_l[0] 
            stages[i]= stages_l[1]
            sigmas.append(s_l)
        if direction == "n":
            stages[i-1]= stages_n[0] 
            stages[i]= stages_n[1]
            sigmas.append(s)
        if direction == "r":
            stages[i-1]= stages_r[0] 
            stages[i]= stages_r[1]
            sigmas.append(s_r)
    return sigmas

In [ ]:
sys_move = system.copy()
sys_move = OutputNormal().apply(sys_move)
sigmas=transform(sys_move.stages,"n")
utils.check_dims(sys_move)

A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

for k in range(1,len(system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas[k-1])]=sigmas[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move = system.copy()
sys_move = OutputNormal().apply(sys_move)
sigmas=transform(sys_move.stages,"r")
utils.check_dims(sys_move)

A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

for k in range(1,len(system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas[k-1])]=sigmas[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
sys_move = system.copy()
sys_move = OutputNormal().apply(sys_move)
sigmas=transform(sys_move.stages,"l")
utils.check_dims(sys_move)

A = matrix

dims_in = sys_move.dims_in
dims_out = sys_move.dims_out

for k in range(1,len(system.stages)):
    sigmas_r = np.linalg.svd(A[-np.sum(dims_out[k:]):,:np.sum(dims_in[:k])],compute_uv=False)
    sigmas_a = np.zeros_like(sigmas_r)
    sigmas_a[:len(sigmas[k-1])]=sigmas[k-1]
    print(np.allclose(sigmas_r,sigmas_a))
    #print("Ref:",sigmas_r)
    #print("calc:",sigmas[k-1])

diff = system.to_matrix()-sys_move.to_matrix()
#plt.matshow(diff)
np.max(np.abs(diff))

In [ ]:
utils.show_system(sys_move)